In [ ]:
import pandas as pd
from datetime import datetime, date
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import os
import pandas as pd
import openpyxl as excel
import glob
import cgmquantify as cgm

In [ ]:
path = r'' # specify data path

extension = 'csv'

os.chdir(path)

files = [i for i in glob.glob('*.{}'.format(extension))]

files

patient_unstrip_IDs = [i.split('_entries', 1)[0] for i in files]

patient_IDs = [ele.lstrip('0') for ele in patient_unstrip_IDs]

#f = files[0]
#print(f)

print(patient_IDs)

df_id_files = pd.DataFrame(
    {'id': patient_IDs,
     'EntriesFileName': files
    })

print(df_id_files)

In [ ]:
all_data = pd.DataFrame()
i = 0

for f in files:
    df = pd.read_csv(f, index_col=None, na_values = " null")
    df = df.rename(columns={"glucose": patient_IDs[i]})
    df = df[patient_IDs[i]]
    all_data = all_data.append(df)
    i=i+1
print(all_data)

In [ ]:
copydata = all_data.T 

In [ ]:
copydata = copydata.dropna()

copydata1 = copydata.apply(pd.to_numeric).T

copydata2 = copydata1.iloc[: , :123]
copydata2

In [ ]:
# Replace infinite updated data with nan
copydata.replace([np.nan, -np.inf], 0, inplace=True)

In [ ]:
from colorsys import hls_to_rgb
import numpy as np
from scipy.cluster.hierarchy import ward, fcluster
import numpy as np

from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn.datasets import load_iris
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage


def fancy_dendrogram(*args, **kwargs):
    max_d = kwargs.pop('max_d', None)
    if max_d and 'color_threshold' not in kwargs:
        kwargs['color_threshold'] = max_d
    annotate_above = kwargs.pop('annotate_above', 0)

    ddata = dendrogram(*args, **kwargs)

    if not kwargs.get('no_plot', False):
        plt.title('Hierarchical Clustering Dendrogram (truncated)')
        plt.xlabel('sample index or (cluster size)')
        plt.ylabel('distance')
        for i, d, c in zip(ddata['icoord'], ddata['dcoord'],
                           ddata['color_list']):
            x = 0.5 * sum(i[1:3])
            y = d[1]
            if y > annotate_above:
                plt.plot(x, y, 'o', c=c)
                plt.annotate("%.3g" % y, (x, y),
                             xytext=(0, -5),
                             textcoords='offset points',
                             va='top',
                             ha='center')
        if max_d:
            plt.axhline(y=max_d, c='k')
    return ddata


def get_distinct_colors(n):

    colors = []

    for i in np.arange(0., 360., 360. / n):
        h = i / 360. + 50
        l = (50 + np.random.rand() * 10) / 100.
        s = (90 + np.random.rand() * 10) / 100.
        colors.append(hls_to_rgb(h, l, s))

    return colors


copydata2 = copydata2.apply(pd.to_numeric).T

X = copydata2.values

print(X)
# generate the linkage matrix
# Perform hierarchical/agglomerative clustering
Z = linkage(X, 'ward')

max_d = 0.7  # max_d as in max_distance

results = pd.DataFrame(index=copydata2.index, columns=['hier'])
# create flat clusters
results['hier'] = fcluster(Z, max_d, criterion='distance')

print(results['hier'])

from matplotlib.cm import get_cmap

groups = results.groupby(['hier'])
name = "tab20"
cmap = get_cmap(name)  # type: matplotlib.colors.ListedColormap
colors = get_distinct_colors(len(groups))  # type: list
k = 0
for name, group in groups:
    groupnames = (group.index.values)
    print('=' * 120)
    print(name)
    fig, ax = plt.subplots(len(groupnames),
                           1,
                           figsize=(20, 5 * len(groupnames)))
    color = colors[k]
    k += 1
    i = 0
    for eachname in groupnames:
        if len(groupnames) == 1:
            ax.plot(all_data[eachname], c=color)
            ax.set_title(eachname)
        else:
            ax[i].plot(all_data[eachname], c=color)
            ax[i].set_title(eachname)
            i += 1
    #plt.show()